In [10]:
import numpy as np
import tensorflow as tf

from pandas import read_excel

from matplotlib import pyplot

#does it make any difference?
%matplotlib inline

#Loading the raw dataset (ignore the first 2 columns A & B)
dataset = read_excel("challenge_dataset.xlsx", usecols="C:Q")

values = dataset.values

# Lets print the first 3 rows of the dataset
print(dataset.head(3))

#Lets create a quick plot of each series (as a separate subplot) and see what we have
i = 0
Nc = values.shape[1] #number of columns
#pyplot.figure()
for group in range(0,Nc):
    i += 1
#    pyplot.subplot(Nc, 1, i)
#    pyplot.plot(values[:, group])
#    pyplot.title(dataset.columns[group], y=0.5, loc='right')
#    #print("dataset.columns[group]", group, dataset.columns[group])
#pyplot.show()


     bx    by    bz    bl    bm    bn  bmag      vx      vy     vz    vmag  \
0  0.52  3.99 -2.53  0.12 -4.74 -0.33  4.75 -212.49  126.52  94.29  264.67   
1 -0.46  2.83 -0.37  1.47 -2.49 -0.08  2.89 -205.30  121.80  91.70  255.72   
2  0.63  3.69 -2.23  0.10 -4.36 -0.12  4.36 -208.85  119.43  89.61  256.73   

     np    tpar    tper  goal  
0  5.51  141.67  157.74     2  
1  6.51  139.22  154.13     2  
2  6.15  144.39  151.92     2  


In [2]:
# ensuring all the data is float
values = values.astype('float32')
print("values.shape =", values.shape)

# normalizing features (columns 1 to 13) NOTE: column 14 = goal (three classes: 0, 1, 2)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values[:,0:Nc-1])
#scaled = values[:,0:Nc-1]

print("scaled.shape =", scaled.shape)

import pandas as pd
df = pd.DataFrame(scaled)

#print('df:', df.head())
#print('dataset:', dataset.head())


values.shape = (18245, 15)
scaled.shape = (18245, 14)


In [11]:
# split the data into train and test sets
# taking 80% of the data for training
n_samples = 89 #each feature has 89 samples
n_series = int( values.shape[0]/n_samples )
n_train = int(0.80 * n_series) * n_samples

# inputs
train_X = np.array(scaled[:n_train, :])
test_X  = np.array(scaled[n_train:, :])
# outpts (making sure outputs are arrays of integers)
last_column = values.shape[1] - 1 #last column (goal) in he raw dataset
train_Y = np.array(values[:n_train, last_column], dtype=int)
test_Y  = np.array(values[n_train:, last_column], dtype=int)

# Lets convert Y from its current shape (m,1) into a "one-hot representation" (m,3),
# to make it suitable for a softmax classifier.
# Now each row in Y_oh will be a one-hot vector giving the class (0, 1, or 2)
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)]
    return Y
train_Y = convert_to_one_hot(train_Y, C = 3)
test_Y  = convert_to_one_hot(test_Y,  C = 3)

# reshape input data to be 3D [samples, timesteps, features]
print("train_X before reshape:", train_X.shape)
print("test_X  before reshape:", test_X.shape)
train_X = train_X.reshape((int(train_X.shape[0]/n_samples), n_samples, train_X.shape[1], 1))
test_X  = test_X.reshape( (int(test_X.shape[0] /n_samples), n_samples,  test_X.shape[1], 1))
print("train_X.shape =", train_X.shape)
print("test_X.shape  =", test_X.shape, "\n")

print("train_Y before reshape:", train_Y.shape)
print("test_Y  before reshape:", test_Y.shape)
train_Y = train_Y.reshape((int(train_Y.shape[0]/n_samples), n_samples, 3))
test_Y  = test_Y.reshape( (int(test_Y.shape[0] /n_samples), n_samples, 3))
print("train_Y.shape =", train_Y.shape)
print("test_Y.shape  =", test_Y.shape)


train_X before reshape: (14596, 14)
test_X  before reshape: (3649, 14)
train_X.shape = (164, 89, 14, 1)
test_X.shape  = (41, 89, 14, 1) 

train_Y before reshape: (14596, 3)
test_Y  before reshape: (3649, 3)
train_Y.shape = (164, 89, 3)
test_Y.shape  = (41, 89, 3)


In [13]:
#import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.layers import Activation, TimeDistributed, Dropout
from keras.layers import Conv2D, Conv1D, Bidirectional, Flatten
from keras.layers import MaxPooling1D, MaxPooling2D
from keras import optimizers

# Meysam:
input_Shape   = (89, 14, 1)
input_Shape   = (164, 89, 14, 1)
train_X_Shape = (164, 89, 14, 1)

# build CNN/LSTM and train it.
model = Sequential()


model.add(TimeDistributed(Conv2D(16, (3,3), padding='same', kernel_initializer='Orthogonal'), input_shape=input_Shape)) 
model.add(Activation('elu')) #'tanh'?
model.add(TimeDistributed(MaxPooling2D(pool_size=(2,2))))
#model.add(Dropout(0.2))

model.add(TimeDistributed(Conv2D(32, (3,3), padding='same', kernel_initializer='Orthogonal'))) 
model.add(Activation('elu'))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2,2))))
#model.add(Dropout(0.2))

model.add(TimeDistributed(Conv2D(64, (3,3), padding='same', kernel_initializer='Orthogonal'))) 
model.add(Activation('elu'))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2,2))))

model.add(TimeDistributed(Flatten()))

model.add(Conv1D(16, 3, padding='same', kernel_initializer='Orthogonal'))
model.add(Activation('elu'))
model.add(MaxPooling1D(pool_size=8))


model.add(Bidirectional(LSTM( 64, return_sequences= True, kernel_initializer='Orthogonal'))) 
model.add(Activation('elu'))
model.add(Bidirectional(LSTM(128, return_sequences=False, kernel_initializer='Orthogonal'))) 
model.add(Activation('elu'))
model.add(Dense(1, activation='softmax')) #sigmoid

adam = optimizers.Adam(lr=0.005, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0001)

### loss='categorical_crossentropy', 'categorical_accuracy', 'msle'
### metrics=['accuracy'], ['msle', 'mae']
model.compile(optimizer=adam, loss='mean_squared_error', metrics=['accuracy'])

# Meysam


# fit network
history = model.fit(train_X, train_Y, epochs=20, batch_size=32,
                    validation_data=(test_X, test_Y), verbose=2, shuffle=False)
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

#summarize the model
model.summary()

ValueError: Error when checking input: expected time_distributed_17_input to have 5 dimensions, but got array with shape (164, 89, 14, 1)